In [ ]:
import requests
from bs4 import BeautifulSoup

def get_text():
    url = 'https://www.gutenberg.org/files/164/164-h/164-h.htm'
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got a successful response
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Convert the entire HTML content to text
    text = soup.get_text()
    
    # Define the start and end markers
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK TWENTY THOUSAND LEAGUES UNDER THE SEA ***"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK TWENTY THOUSAND LEAGUES UNDER THE SEA ***"
    
    # Find the positions of the start and end markers
    start_pos = text.find(start_marker)
    end_pos = text.find(end_marker)
    
    # Check if both markers are found
    if start_pos == -1 or end_pos == -1:
        return "Markers not found in the text."
    
    # Extract the text between the start and end markers
    extracted_text = text[start_pos+len(start_marker):end_pos].strip()
    
    return extracted_text


def get_biografies() -> str:
    return """Albert Einstein: Albert Einstein was a theoretical physicist born in 1879 in Ulm, Germany. He is best known for developing the theory of relativity, one of the two pillars of modern physics. His equation E=mc², which expresses the equivalence of mass and energy, is one of the most famous equations in the world. Einstein received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect, which was pivotal in establishing quantum theory. He spent his later years at the Institute for Advanced Study in Princeton, New Jersey, and remained a vocal advocate for peace and human rights until his death in 1955.
Galileo Galilei: Galileo Galilei, born in Pisa in 1564, was an Italian polymath who played a major role in the Scientific Revolution. He is renowned for his contributions to astronomy, including the improvement of the telescope and consequent astronomical observations that supported the Copernican system. Galileo discovered the four largest moons of Jupiter, now known as the Galilean moons. He also made significant contributions to physics, particularly in the study of motion. His advocacy of heliocentrism led to his trial and house arrest by the Inquisition, where he spent his final years until his death in 1642.
Christopher Columbus: Christopher Columbus was an Italian explorer and navigator born in 1451 in Genoa. He completed four voyages across the Atlantic Ocean, opening the way for the widespread European exploration and colonization of the Americas. Sponsored by the Catholic Monarchs of Spain, his first voyage in 1492 reached the New World, landing in the Bahamas. Columbus's expeditions, while controversial for their impact on indigenous populations, were pivotal in the Age of Exploration, significantly altering the course of history by connecting the Old World with the New.
Isaac Newton: Sir Isaac Newton, born in 1643 in Woolsthorpe, England, was a mathematician, physicist, and astronomer who is widely recognized as one of the most influential scientists of all time. His work in mathematics includes the development of calculus. In physics, he formulated the laws of motion and universal gravitation, which laid the groundwork for classical mechanics. Newton's book "Principia Mathematica" is considered one of the most important works in the history of science. He also made significant contributions to optics and held the position of Warden of the Royal Mint until his death in 1727.
Francisco de Miranda: Francisco de Miranda, born in 1750 in Caracas, Venezuela, was a revolutionary leader, military commander, and key figure in the struggle for Latin American independence. He is often called "The Precursor" because of his early efforts to liberate Spanish America from colonial rule. Miranda traveled extensively throughout Europe and the Americas, gathering support for the independence cause. He played a significant role in the early stages of the Venezuelan War of Independence and sought to unite all Spanish American colonies into a single confederation. Although his efforts were initially unsuccessful, Miranda's vision and actions laid the groundwork for future independence movements. He was captured by Spanish forces and died in prison in 1816.
Nikola Tesla: Nikola Tesla was a Serbian-American inventor, electrical engineer, and futurist born in 1856. Tesla is best known for his contributions to the development of alternating current (AC) electrical systems, which became the dominant form of electrical transmission. His inventions and theoretical work laid the foundation for modern electrical engineering. Tesla also conducted pioneering work in radio, x-rays, and wireless transmission of energy. Despite his many achievements, he died penniless in 1943, with much of his work unrecognized during his lifetime, though he is now celebrated as a visionary inventor.
Pythagoras: Pythagoras of Samos, born around 570 BCE, was an ancient Greek philosopher and mathematician who founded the Pythagorean school of thought. He is best known for the Pythagorean theorem, which establishes the relationship between the sides of a right triangle. Pythagoras's influence extended beyond mathematics to philosophy and religion; his followers believed in the transmigration of souls and the importance of numerical relationships in understanding the universe. Although much of his life and work remains shrouded in legend, his contributions to mathematics and philosophy have had a lasting impact.
Alexandre Dumas: Alexandre Dumas, born in 1802 in France, was a prolific writer known for his historical novels of high adventure. His most famous works include "The Three Musketeers," "Twenty Years After," and "The Count of Monte Cristo." Dumas's novels are celebrated for their vivid characters, intricate plots, and sense of adventure. He also wrote plays, essays, and travel books. Dumas's works have been translated into numerous languages and adapted into countless films and television series, securing his place as one of the most widely read French authors. He died in 1870, leaving a vast literary legacy.
Simón Bolívar: Simón Bolívar, born in 1783 in Caracas, Venezuela, was a military and political leader who played a key role in Latin America's successful struggle for independence from the Spanish Empire. Known as "El Libertador," Bolívar led military campaigns that resulted in the liberation of Venezuela, Bolivia, Colombia, Ecuador, Peru, and Panama. He envisioned the creation of a united Latin American state, Gran Colombia, but internal conflicts and political differences eventually led to its dissolution. Bolívar's legacy endures as a symbol of liberation and unity in Latin America.
Napoleon Bonaparte: Napoleon Bonaparte, born in 1769 in Corsica, was a French military and political leader who rose to prominence during the French Revolution and its associated wars. He became Emperor of the French from 1804 until 1814, and again briefly in 1815 during the Hundred Days. Napoleon is best known for his Napoleonic Code, which influenced legal systems worldwide, and his military campaigns, which led to significant territorial expansion. His defeat at the Battle of Waterloo in 1815 marked the end of his rule. Napoleon was exiled to the island of Saint Helena, where he died in 1821.
Henry VIII: Henry VIII, born in 1491, was King of England from 1509 until his death in 1547. He is best known for his six marriages and his role in the separation of the Church of England from the Roman Catholic Church. Henry's desire for an annulment from his first wife, Catherine of Aragon, led to a conflict with the Pope, resulting in the English Reformation. He established himself as the Supreme Head of the Church of England, leading to significant religious and political changes. Henry's reign was marked by his efforts to consolidate royal power and his complex personal life.
Julius Caesar: Gaius Julius Caesar, born in 100 BCE, was a Roman general, statesman, and dictator who played a critical role in the events that led to the demise of the Roman Republic and the rise of the Roman Empire. His military campaigns expanded Rome's territories, notably in Gaul. Caesar's crossing of the Rubicon River ignited a civil war, which he ultimately won, leading to his appointment as dictator for life. His reforms and centralization of power angered many in the Senate, leading to his assassination on the Ides of March in 44 BCE. Caesar's legacy profoundly shaped Roman history.
Cleopatra: Cleopatra VII Philopator, born in 69 BCE, was the last active ruler of the Ptolemaic Kingdom of Egypt. Renowned for her intelligence, political acumen, and romantic liaisons with Roman leaders Julius Caesar and Mark Antony, Cleopatra sought to protect Egypt's independence amidst the expanding Roman Empire. Her relationships with Caesar and Antony were both politically and personally motivated, aiming to secure her position and her kingdom's future. Cleopatra's life and reign ended in 30 BCE, following Antony's defeat at the Battle of Actium. Her dramatic death marked the end of Ptolemaic rule in Egypt.
George Washington: George Washington, born in 1732 in Westmoreland County, Virginia, was the first President of the United States, serving from 1789 to 1797. He is often referred to as the "Father of His Country" for his pivotal role in the founding of the United States. Washington served as the Commander-in-Chief of the Continental Army during the American Revolutionary War, leading American forces to victory against Great Britain. His leadership and strategic acumen were instrumental in securing American independence. Following the war, Washington presided over the Constitutional Convention of 1787, which led to the drafting of the U.S. Constitution.
Montezuma II: Montezuma II, born around 1466, was the ninth emperor of the Aztec Empire, reigning from 1502 to 1520. Under his rule, the Aztec Empire reached its greatest territorial extent. Montezuma is best known for his encounter with the Spanish conquistadors led by Hernán Cortés. Initially welcoming the Spaniards, Montezuma was later taken hostage, and his leadership during this critical period remains a subject of historical debate. His reign ended amidst the Spanish conquest of the Aztec capital, Tenochtitlán. Montezuma's legacy is intertwined with the dramatic fall of the Aztec Empire.
Atahualpa: Atahualpa, born in 1497, was the last emperor of the Inca Empire. He came to power after a civil war with his half-brother Huáscar. Atahualpa's reign was abruptly ended when he was captured by Spanish conquistador Francisco Pizarro in 1532. Despite his attempts to secure his release through a large ransom of gold and silver, Atahualpa was executed by the Spaniards in 1533. His death marked the beginning of the end for the Inca Empire, leading to its rapid decline and the expansion of Spanish influence in South America.
Henry Ford: Henry Ford, born in 1863, was an American industrialist and founder of the Ford Motor Company. He revolutionized the automotive industry by introducing the Model T, the first affordable automobile, and pioneering assembly line production techniques. Ford's innovations significantly lowered the cost of manufacturing and made cars accessible to a broader population. His vision extended beyond manufacturing; he implemented the $5 workday, doubling wages and reducing working hours, which helped to create a middle-class consumer base. Ford's impact on industrial production and labor practices is profound and enduring.
Ford Motor Company: Founded by Henry Ford in 1903, the Ford Motor Company is an American multinational automaker headquartered in Dearborn, Michigan. Ford revolutionized the automotive industry with the introduction of the Model T and the development of assembly line manufacturing, which drastically reduced production costs and made cars affordable for the average consumer. Over the years, Ford has produced numerous iconic vehicles and has remained a significant player in the global automotive market. The company's commitment to innovation continues with advancements in electric and autonomous vehicles.
Mercedes-Benz: Mercedes-Benz is a German luxury automotive brand known for producing high-quality vehicles, including luxury cars, buses, and trucks. The brand's origins date back to 1886 when Karl Benz created the first petrol-powered car, the Benz Patent-Motorwagen. Mercedes-Benz was officially founded in 1926 with the merger of Karl Benz's and Gottlieb Daimler's companies. Renowned for its engineering excellence and innovation, Mercedes-Benz has introduced numerous safety and performance features that have become industry standards. The brand continues to be synonymous with luxury, performance, and cutting-edge automotive technology.
Nestlé: Nestlé is a Swiss multinational food and beverage company founded in 1867 by Henri Nestlé. Headquartered in Vevey, Switzerland, Nestlé is the largest food company in the world by revenue. Its product portfolio includes baby food, bottled water, breakfast cereals, coffee and tea, dairy products, ice cream, frozen food, pet foods, and snacks. Nestlé is also known for its commitment to nutrition, health, and wellness. The company has made significant efforts to improve the sustainability of its supply chain and reduce its environmental impact.
NASA: The National Aeronautics and Space Administration (NASA) is the United States government agency responsible for the nation's civilian space program and for aeronautics and aerospace research. Established in 1958, NASA has led numerous space exploration missions, including the Apollo moon-landing missions, the Skylab space station, and the Space Shuttle program. NASA's current initiatives include the Artemis program, which aims to return humans to the Moon, and the Mars Exploration Program, which seeks to explore the Red Planet. NASA's scientific research has significantly advanced our understanding of the universe.
United Nations (UN): The United Nations (UN) is an international organization founded in 1945 after World War II to promote peace, security, and cooperation among member states. Headquartered in New York City, the UN has 193 member countries. Its main organs include the General Assembly, the Security Council, the International Court of Justice, and specialized agencies such as WHO and UNESCO. The UN addresses global issues such as human rights, environmental protection, humanitarian aid, and sustainable development. Its peacekeeping missions aim to resolve conflicts and support stability worldwide.
NATO: The North Atlantic Treaty Organization (NATO) is a military alliance established in 1949 to provide collective security against the Soviet Union. NATO's founding members include the United States, Canada, and several Western European nations. The organization's core principle is mutual defense, whereby an armed attack against one member is considered an attack against all members. NATO's role has evolved since the Cold War, with its operations now including crisis management, cooperative security, and counterterrorism. NATO continues to adapt to new security challenges in the 21st century.
Thomas Edison: Thomas Edison, born in 1847, was an American inventor and businessman known for his numerous contributions to technology and industry. Edison's most famous inventions include the phonograph, the motion picture camera, and the practical incandescent light bulb. He held over 1,000 patents and established the first industrial research laboratory, Menlo Park, which laid the foundation for modern R&D practices. Edison's work in electric power generation and distribution played a crucial role in the widespread adoption of electricity. He passed away in 1931, leaving a legacy of innovation.
Marie Curie: Marie Curie, born in 1867 in Warsaw, Poland, was a physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and remains the only person to win Nobel Prizes in two different scientific fields (Physics in 1903 and Chemistry in 1911). Curie's discoveries, including the elements polonium and radium, advanced the understanding of radioactivity and its applications. Her work laid the groundwork for the development of x-ray machines. Curie died in 1934 from aplastic anemia, likely caused by her prolonged exposure to radiation.
Wright Brothers: Orville and Wilbur Wright, born in 1871 and 1867 respectively, were American inventors and aviation pioneers who achieved the first controlled, sustained flight of a powered, heavier-than-air aircraft on December 17, 1903, in Kitty Hawk, North Carolina. The Wright brothers' meticulous research and development of the airplane, including their innovative three-axis control system, revolutionized transportation and paved the way for modern aviation. Their successful flights marked a significant milestone in human history, leading to advancements in aeronautics and the eventual development of commercial aviation.
Alexander Graham Bell: Alexander Graham Bell, born in 1847, was a Scottish-born inventor, scientist, and teacher of the deaf. He is best known for inventing the telephone in 1876, a breakthrough that revolutionized global communication. Bell's other notable contributions include advancements in optical telecommunications, aeronautics, and hydrofoils. He co-founded the American Telephone and Telegraph Company (AT&T) in 1885. Bell's work with the deaf community and his innovative spirit had a profound impact on multiple fields. He passed away in 1922, leaving a legacy of ingenuity and invention.
Sigmund Freud: Sigmund Freud, born in 1856, was an Austrian neurologist and the founder of psychoanalysis, a clinical method for treating psychopathology through dialogue between a patient and a psychoanalyst. Freud developed influential theories about the unconscious mind, including the concepts of the id, ego, and superego. His ideas on childhood sexuality, dream interpretation, and the mechanisms of repression have had a lasting impact on psychology, psychiatry, and the humanities. Freud's work, though controversial, established him as one of the most influential thinkers of the 20th century. He died in 1939.
Martin Luther King Jr.: Martin Luther King Jr., born in 1929, was an American Baptist minister and civil rights leader who played a pivotal role in the American civil rights movement. He advocated for nonviolent resistance to racial segregation and discrimination. King's leadership in events such as the Montgomery Bus Boycott and the March on Washington, where he delivered his famous "I Have a Dream" speech, helped to bring about significant legislative changes, including the Civil Rights Act of 1964. King was awarded the Nobel Peace Prize in 1964. He was assassinated in 1968, leaving a lasting legacy of equality and justice.
Walt Disney: Walt Disney, born in 1901, was an American entrepreneur, animator, and film producer who co-founded The Walt Disney Company. Disney created iconic characters such as Mickey Mouse and pioneered animated feature films with works like "Snow White and the Seven Dwarfs." He expanded into theme parks with the opening of Disneyland in 1955. Disney's innovative spirit and creative vision transformed the entertainment industry, making his name synonymous with family entertainment. He won 22 Academy Awards during his career. Disney passed away in 1966, but his legacy continues to influence popular culture.
Pablo Picasso: Pablo Picasso, born in 1881 in Spain, was a painter, sculptor, printmaker, ceramicist, and one of the most influential artists of the 20th century. Picasso co-founded the Cubist movement and made significant contributions to Surrealism and Symbolism. His works are characterized by their stylistic diversity, ranging from the Blue and Rose Periods to the development of Cubism with Georges Braque. Iconic works such as "Les Demoiselles d'Avignon" and "Guernica" reflect his innovative approach and social commentary. Picasso's prolific career spanned over seven decades until his death in 1973.
Bartolomeu Dias: Bartolomeu Dias, born around 1450 in Portugal, was a pioneering Portuguese explorer. In 1488, Dias became the first European to sail around the southern tip of Africa, later named the Cape of Good Hope. This monumental voyage proved that the Atlantic and Indian Oceans were connected, opening a new sea route for trade with Asia. Dias's expedition laid the groundwork for future explorations by Vasco da Gama and others. Despite the challenges and dangers faced during his journey, Dias's achievements were instrumental in expanding Portuguese exploration. He died in 1500 during a voyage led by Pedro Álvares Cabral.
Mother Teresa: Mother Teresa, born in 1910 as Anjezë Gonxhe Bojaxhiu in Macedonia, was a Catholic nun and missionary known for her work with the poor in Kolkata, India. She founded the Missionaries of Charity in 1950, a religious congregation dedicated to helping those in need. Mother Teresa's humanitarian efforts earned her numerous accolades, including the Nobel Peace Prize in 1979. She was canonized as Saint Teresa of Calcutta in 2016. Her life and work remain an inspiration to many for their emphasis on compassion and service to humanity. She passed away in 1997.
Charles Darwin: Charles Darwin, born in 1809, was an English naturalist and geologist best known for his contributions to the theory of evolution. His seminal work, "On the Origin of Species" published in 1859, introduced the concept of natural selection as the mechanism by which evolution occurs. Darwin's extensive research during the voyage of the HMS Beagle and his subsequent studies provided compelling evidence for the interrelatedness of all life forms. His ideas revolutionized biological science and had profound implications for various fields, including genetics, ecology, and anthropology. Darwin passed away in 1882.
Steve Jobs: Steve Jobs, born in 1955, was an American entrepreneur and co-founder of Apple Inc. He is widely recognized for his role in revolutionizing the technology industry through innovations in personal computing, mobile phones, and digital media. Under his leadership, Apple introduced groundbreaking products such as the Macintosh, iPod, iPhone, and iPad. Jobs also co-founded Pixar Animation Studios, contributing to the advancement of computer-animated films. His vision and creativity transformed multiple industries and left a lasting impact on global technology and design. Jobs passed away in 2011.
Google: Google LLC, founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University, is an American multinational technology company specializing in Internet-related services and products. Google's core product, the search engine, has become the most widely used search tool globally. The company's portfolio includes various products and services such as Google Maps, Gmail, Android, and YouTube. Google's mission is to organize the world's information and make it universally accessible and useful. The company is known for its significant contributions to technology, innovation, and digital advertising.
Amazon: Amazon, founded by Jeff Bezos in 1994, is an American multinational technology company based in Seattle, Washington. Initially an online bookstore, Amazon expanded its product offerings to become the world's largest online retailer and a major provider of cloud computing services through Amazon Web Services (AWS). The company has also ventured into consumer electronics with products like the Kindle, Echo, and Fire TV. Amazon's influence extends to various sectors, including e-commerce, digital streaming, and artificial intelligence. Its customer-centric approach and innovation have revolutionized global retail and technology industries.
IBM: International Business Machines Corporation (IBM), founded in 1911, is an American multinational technology and consulting company headquartered in Armonk, New York. Originally known for its tabulating machines, IBM evolved into a major player in computer hardware, software, and services. The company's notable contributions include the development of the mainframe computer, the IBM PC, and advancements in artificial intelligence with IBM Watson. IBM's research and innovation have played a critical role in the evolution of technology. The company continues to focus on cloud computing, cognitive computing, and data analytics.
Intel: Intel Corporation, founded in 1968 by Robert Noyce and Gordon Moore, is an American multinational corporation and technology company headquartered in Santa Clara, California. Intel is the world's largest and highest-valued semiconductor chip manufacturer. The company's innovations in microprocessor technology have driven the development of personal computers, servers, and data centers. Intel's processors power a vast majority of the world's computers, making it a cornerstone of the modern computing industry. The company's ongoing research and development efforts continue to push the boundaries of technology and innovation.
Samsung: Samsung, founded in 1938 by Lee Byung-chul, is a South Korean multinational conglomerate headquartered in Seoul. Initially a trading company, Samsung diversified into various industries, becoming a global leader in electronics, particularly in smartphones, televisions, and semiconductors. Samsung Electronics, the flagship subsidiary, is one of the world's largest technology companies by revenue. The company's innovative products and significant investments in research and development have solidified its position as a major player in the global technology market. Samsung continues to influence multiple industries, from consumer electronics to construction and finance.
Apple Inc.: Apple Inc., founded in 1976 by Steve Jobs, Steve Wozniak, and Ronald Wayne, is an American multinational technology company headquartered in Cupertino, California. Apple is renowned for its innovative products, including the Macintosh computer, iPod, iPhone, iPad, and MacBook. The company's software offerings include the macOS, iOS, watchOS, and tvOS operating systems. Apple's emphasis on design, user experience, and ecosystem integration has garnered a loyal customer base and made it one of the most valuable companies in the world. Apple's influence extends across technology, entertainment, and digital communication.
Microsoft: Microsoft Corporation, founded in 1975 by Bill Gates and Paul Allen, is an American multinational technology company headquartered in Redmond, Washington. Microsoft is best known for its Windows operating system, Office suite of productivity software, and Azure cloud computing platform. The company's innovative products and services have shaped the personal computing industry and influenced various sectors, including enterprise software, gaming (with Xbox), and artificial intelligence. Microsoft's commitment to innovation, research, and development has solidified its position as a global technology leader.
Facebook: Facebook, Inc., founded in 2004 by Mark Zuckerberg and his Harvard College roommates, is an American social media and technology company headquartered in Menlo Park, California. Facebook's platform revolutionized social networking, allowing users to connect, share, and communicate with friends and family worldwide. The company's growth led to the acquisition of other major platforms such as Instagram and WhatsApp. Despite facing challenges related to privacy and data security, Facebook remains one of the most influential and widely used social media networks globally, with significant impact on digital communication and advertising.
General Electric (GE): General Electric Company (GE), founded in 1892, is an American multinational conglomerate headquartered in Boston, Massachusetts. GE operates in various sectors, including aviation, healthcare, power, renewable energy, and digital technology. The company's innovations have historically included advancements in lighting, power generation, and medical imaging. GE's commitment to research and development has driven technological progress across multiple industries. Despite challenges in recent years, GE continues to be a significant player in industrial manufacturing and technological innovation.
Vasco da Gama: Vasco da Gama, born around 1460 in Sines, Portugal, was a prominent Portuguese explorer. He is best known for being the first European to reach India by sea, linking Europe and Asia via the Cape of Good Hope. Da Gama's voyages, beginning in 1497, were crucial in establishing a maritime route to the East, which significantly boosted trade between Europe and Asia. His successful expeditions paved the way for the Portuguese colonial empire in Asia. Vasco da Gama was appointed Viceroy of India in 1524, but he died shortly after arriving in Cochin. His contributions to exploration and navigation are celebrated in Portuguese history.
Sony: Sony Corporation, founded in 1946 by Masaru Ibuka and Akio Morita, is a Japanese multinational conglomerate headquartered in Tokyo. Sony is known for its contributions to consumer electronics, gaming, entertainment, and financial services. The company's iconic products include the Walkman, PlayStation, and Bravia televisions. Sony's film and music divisions have produced and distributed numerous successful films, television shows, and music albums. The company's commitment to innovation and quality has established it as a leading global brand in technology and entertainment.
PepsiCo: PepsiCo, Inc., founded in 1965 through the merger of Pepsi-Cola Company and Frito-Lay, Inc., is an American multinational food and beverage corporation headquartered in Purchase, New York. The company's diverse product portfolio includes popular brands such as Pepsi, Mountain Dew, Lay's, Doritos, Tropicana, and Quaker. PepsiCo's commitment to innovation, marketing, and sustainability has made it a leader in the global food and beverage industry. The company's focus on health and wellness, environmental responsibility, and community engagement continues to drive its growth and influence.
Coca-Cola: The Coca-Cola Company, founded in 1886 by John Stith Pemberton, is an American multinational beverage corporation headquartered in Atlanta, Georgia. Coca-Cola's flagship product, the Coca-Cola soft drink, is one of the most recognized and widely consumed beverages worldwide. The company's extensive product portfolio includes various soft drinks, juices, teas, and bottled water. Coca-Cola's global marketing strategies, innovative packaging, and commitment to sustainability have solidified its position as a leading beverage company. The brand's cultural impact and iconic status continue to shape the beverage industry.
Andrés Bello: Andrés Bello, born in 1781 in Caracas, Venezuela, was a distinguished intellectual, poet, lawmaker, and educator in Latin America. He played a significant role in the cultural and intellectual development of Spanish America during the 19th century. Bello's contributions include the establishment of the University of Chile, where he served as its first rector, and the development of the Chilean Civil Code, which influenced other Latin American legal systems. He was also a prolific writer and linguist, contributing extensively to the study of Spanish grammar and literature. Bello's work in education, law, and literature left a lasting legacy on Latin American intellectual history. He passed away in 1865 in Santiago, Chile.
Toyota: Toyota Motor Corporation, founded in 1937 by Kiichiro Toyoda, is a Japanese multinational automotive manufacturer headquartered in Toyota City, Japan. Toyota is known for producing reliable, fuel-efficient vehicles and pioneering innovations in hybrid technology with the introduction of the Prius. The company's commitment to quality, continuous improvement (kaizen), and sustainable practices has made it one of the largest and most respected automotive brands globally. Toyota's emphasis on research and development continues to drive advancements in electric and autonomous vehicles.
Amazon Web Services (AWS): Amazon Web Services (AWS), launched in 2006 by Amazon.com, Inc., is a comprehensive and widely adopted cloud platform offering over 200 fully-featured services from data centers globally. AWS provides on-demand cloud computing platforms and APIs to individuals, companies, and governments, on a metered pay-as-you-go basis. Its services include computing power, storage, and databases, making it a critical component of modern digital infrastructure. AWS's scalability, flexibility, and reliability have established it as a leader in the cloud computing industry, supporting businesses of all sizes in their digital transformation efforts.
Intel Corporation: Intel Corporation, founded in 1968 by Robert Noyce and Gordon Moore, is an American multinational corporation and technology company headquartered in Santa Clara, California. Intel is the world's largest and highest-valued semiconductor chip manufacturer. The company's innovations in microprocessor technology have driven the development of personal computers, servers, and data centers. Intel's processors power a vast majority of the world's computers, making it a cornerstone of the modern computing industry. The company's ongoing research and development efforts continue to push the boundaries of technology and innovation.
Leonardo da Vinci: Leonardo da Vinci, born in 1452 in Vinci, Italy, was a polymath whose areas of interest included painting, sculpture, architecture, science, music, mathematics, engineering, and anatomy. Known as the quintessential Renaissance man, his masterpieces include "The Last Supper" and "Mona Lisa." Da Vinci's notebooks, filled with sketches and scientific diagrams, reveal his extensive studies in anatomy, mechanics, and nature. His innovative designs, although not realized in his lifetime, anticipated many modern technologies. Da Vinci's work continues to inspire countless artists and scientists. He died in 1519.
Johann Sebastian Bach: Johann Sebastian Bach, born in 1685 in Eisenach, Germany, was a composer and musician of the Baroque period. Renowned for his mastery of counterpoint, harmonic and motivic organization, and adaptation of rhythms and forms, Bach's works include the "Brandenburg Concertos," "The Well-Tempered Clavier," and the "Mass in B minor." His music is celebrated for its technical command and artistic beauty. Although he was not widely recognized during his lifetime, Bach's music has since become a cornerstone of classical repertoire. He died in 1750.
Ada Lovelace: Ada Lovelace, born in 1815 in London, England, was a mathematician and writer, chiefly known for her work on Charles Babbage's early mechanical general-purpose computer, the Analytical Engine. She is often considered the first computer programmer due to her creation of an algorithm intended to be processed by a machine. Lovelace's notes on the Analytical Engine include what is recognized as the first algorithm designed for implementation on a computer. Her visionary insights into the potential of computing have earned her a lasting legacy in the history of computing. She died in 1852.
Nelson Mandela: Nelson Mandela, born in 1918 in Mvezo, South Africa, was a revolutionary anti-apartheid leader who became South Africa's first black president from 1994 to 1999. Imprisoned for 27 years for his opposition to apartheid, Mandela emerged as a symbol of resistance and reconciliation. His presidency marked the end of institutionalized racial segregation and ushered in a new era of democratic governance. Mandela was awarded the Nobel Peace Prize in 1993. His lifelong dedication to justice, equality, and human rights has made him a global icon. He passed away in 2013.
Warren Buffett: Warren Buffett, born in 1930 in Omaha, Nebraska, is an American business magnate, investor, and philanthropist. He is the chairman and CEO of Berkshire Hathaway, a multinational conglomerate holding company. Known as the "Oracle of Omaha," Buffett is one of the most successful investors of all time, with a net worth consistently ranking him among the world's wealthiest individuals. His investment philosophy emphasizes value investing and long-term growth. Buffett has pledged to give away the majority of his fortune to philanthropic causes, primarily through the Bill & Melinda Gates Foundation.
Guglielmo Marconi: Guglielmo Marconi, born in 1874 in Bologna, Italy, was an inventor and electrical engineer known for his pioneering work on long-distance radio transmission and for developing Marconi's law and a radio telegraph system. He is credited with the invention of the radio, which he successfully demonstrated in 1895. Marconi's innovations earned him the Nobel Prize in Physics in 1909, shared with Karl Ferdinand Braun. His work laid the foundation for modern wireless communication. Marconi continued to advance radio technology until his death in 1937.
The Boeing Company: The Boeing Company, founded in 1916 by William E. Boeing in Seattle, Washington, is one of the largest aerospace and defense manufacturers in the world. Boeing produces commercial jetliners, military aircraft, satellites, and space exploration vehicles. Its iconic products include the 747 jumbo jet and the 787 Dreamliner. Boeing's innovations have significantly shaped the aviation industry, enhancing global connectivity and advancing aerospace technology. Despite facing challenges, Boeing remains a key player in the aerospace sector, driving advancements in both commercial and defense aviation.
Amelia Earhart: Amelia Earhart, born in 1897 in Atchison, Kansas, was an American aviation pioneer and author. She was the first female aviator to fly solo across the Atlantic Ocean, achieving this feat in 1932. Earhart's courage and determination made her an international icon of women's achievement. She also set numerous other aviation records and authored best-selling books about her flying experiences. In 1937, during an attempt to circumnavigate the globe, Earhart disappeared over the central Pacific Ocean. Her disappearance remains one of the greatest unsolved mysteries in aviation history.
Real Compañía Guipuzcoana de Caracas: The Real Compañía Guipuzcoana de Caracas, commonly known as the Gipuzkoan Company of Caracas, was a Spanish trading company established in 1728 by the Basque province of Gipuzkoa. The company's primary goal was to promote and monopolize trade between Spain and its colony, Venezuela, particularly the export of cocoa. The establishment of the company aimed to curb the rampant smuggling and ensure that the Spanish Crown benefitted from the lucrative cocoa trade. The Guipuzcoana Company built infrastructure, such as warehouses and ports, and fostered agricultural development in Venezuela.
Mahatma Gandhi: Mohandas Karamchand Gandhi, born in 1869 in Porbandar, India, was a leader of the Indian independence movement against British colonial rule. Known as Mahatma (meaning "great soul"), Gandhi advocated for nonviolent resistance and civil disobedience as powerful tools for social and political change. His leadership led to significant movements such as the Salt March and the Quit India Movement. Gandhi's philosophy of nonviolence influenced civil rights movements worldwide. He was assassinated in 1948, but his legacy of peace and justice continues to inspire global movements for social change.
Vatican City: Vatican City, an independent city-state enclaved within Rome, Italy, is the spiritual and administrative center of the Roman Catholic Church. Established in 1929 with the Lateran Treaty, it is the smallest independent state in the world by both area and population. Vatican City is home to the Pope, the leader of the Catholic Church, and serves as the location for the central governing body of the Church. The city-state contains significant religious and cultural sites, including St. Peter's Basilica, the Sistine Chapel, and the Vatican Museums, which house a vast collection of art and historical artifacts.
Ludwig van Beethoven: Ludwig van Beethoven, born in 1770 in Bonn, Germany, was a composer and pianist who remains one of the most admired figures in Western classical music. His works span the transition from the Classical to the Romantic era, and his compositions are known for their emotional depth and technical mastery. Notable works include his nine symphonies, the "Moonlight Sonata," and "Fidelio," his only opera. Despite losing his hearing later in life, Beethoven continued to compose some of his most important works. He passed away in 1827, leaving a lasting legacy in music.
Alberto Santos-Dumont: Alberto Santos-Dumont, born in 1873 in Palmira, Brazil, was a pioneering aviator and inventor. He is best known for his significant contributions to the development of aviation. Santos-Dumont designed, built, and flew the first dirigible that could be steered and controlled with precision, winning the Deutsch de la Meurthe prize in 1901. In 1906, he achieved a major milestone in aviation history by piloting the 14-bis, a heavier-than-air aircraft, for a distance of 220 meters (722 feet) in Paris, becoming the first person to make a public flight in a powered aircraft. Santos-Dumont's innovations and public demonstrations helped to advance the field of aviation and inspired future generations of aviators. He is celebrated as a national hero in Brazil and a key figure in the early history of flight. Santos-Dumont passed away in 1932, but his legacy continues to inspire aviation enthusiasts worldwide.
The Red Cross: The International Red Cross and Red Crescent Movement is a humanitarian institution founded in 1863 in Geneva, Switzerland, by Henry Dunant and Gustave Moynier. It provides emergency assistance, disaster relief, and education in communities affected by war and natural disasters. The Red Cross operates under the principles of neutrality, impartiality, and humanity, aiming to protect human life and health. It consists of the International Committee of the Red Cross (ICRC), the International Federation of Red Cross and Red Crescent Societies (IFRC), and numerous national societies. Its efforts have earned global recognition, including multiple Nobel Peace Prizes.
Pedro Álvares Cabral: Pedro Álvares Cabral, born in 1467 or 1468 in Belmonte, Portugal, was a notable Portuguese navigator and explorer. He is best known for leading the expedition that resulted in the discovery of Brazil in 1500. Commanding a fleet sent to India, Cabral's ships were blown off course, leading to the accidental discovery of the Brazilian coast. He claimed the land for Portugal, marking the beginning of Portuguese influence in South America. Cabral continued his voyage to India, where he established trade relations and strengthened Portugal's presence in the Indian Ocean. His contributions to exploration were significant in expanding Portuguese territories and trade networks.
Antonio José de Sucre: Antonio José de Sucre, born in 1795 in Cumaná, Venezuela, was a prominent military leader and statesman during the Latin American wars of independence. A close ally of Simón Bolívar, Sucre played a pivotal role in the liberation of South America from Spanish rule. He is best known for his decisive victory at the Battle of Ayacucho in 1824, which effectively ended Spanish dominion in South America. Sucre went on to serve as the second President of Bolivia and held various important political and military positions. Revered for his military prowess and integrity, Sucre's legacy as a liberator and national hero is celebrated throughout Latin America. He was assassinated in 1830 in Colombia.
"""


In [2]:
def get_coldf_experiment_text():
    return """[Experiment 1]
[May 23, 2024]
The first experiment focused on using palladium electrodes submerged in heavy water (deuterium oxide, D2O). Dr. Emily D. Jensen, Senior Physicist, led this trial. The procedure involved electrolysis at a constant current of 50 mA, aiming to induce cold fusion within the palladium lattice. Throughout the 12-hour process, temperatures were carefully monitored, maintaining a steady 25°C. Voltage readings were recorded every hour to observe any anomalies indicating fusion events. The experiment yielded promising preliminary results with minor heat generation detected, suggesting potential excess energy beyond chemical reactions.
To ensure accuracy, additional measurements included the analysis of gas output. Both hydrogen and deuterium gas levels were monitored using a gas chromatograph, which revealed an increase in deuterium gas concentration over time. This increase suggested that deuterium nuclei might be fusing within the palladium lattice. Further isotopic analysis using mass spectrometry confirmed the presence of helium-4, a potential byproduct of fusion, indicating that cold fusion might indeed be occurring.
[Experiment 2]
[May 24, 2024]
Dr. Samuel C. Lee, Junior Researcher, conducted the second experiment, replacing palladium with nickel electrodes. The setup involved similar electrolysis in heavy water but at a higher current of 100 mA. Temperatures were initially set at 30°C, increasing to 40°C after 6 hours. Measurements of voltage, current, and temperature were taken every 30 minutes to ensure precision. Despite the increased parameters, the experiment showed minimal heat excess, indicating nickel might not be as effective as palladium in this configuration. Further analysis was needed to understand the material interactions fully.
In addition to temperature and voltage monitoring, neutron detection was employed using a neutron scintillation detector. The results showed sporadic neutron emissions, which could be indicative of fusion reactions. However, the neutron counts were significantly lower compared to those observed with palladium electrodes, suggesting a lower fusion rate. Post-experiment analysis of the nickel electrodes using scanning electron microscopy (SEM) revealed microstructural changes, indicating that some level of nuclear interaction might have occurred.
[Experiment 3]
[May 25, 2024]
The third experiment, overseen by Dr. Karen H. Patel, Chief Scientist, introduced lithium into the electrolyte solution, aiming to enhance deuterium absorption in the palladium electrodes. The electrolysis current was set at 75 mA, and the temperature was maintained at 35°C. Voltage and temperature readings were taken every 15 minutes. After 10 hours, there was a noticeable increase in heat output, suggesting a successful cold fusion reaction. The lithium additive appeared to play a significant role in the process, warranting further exploration.
Detailed calorimetric analysis revealed that the excess heat produced was consistent and reproducible. The energy output exceeded the input by approximately 15%, a significant margin indicating a potential cold fusion reaction. Additionally, spectroscopy of the electrolyte solution showed traces of helium-4, further supporting the occurrence of fusion. The palladium electrodes, post-experiment, exhibited increased deuterium content, confirming enhanced absorption facilitated by the lithium additive.
[Experiment 4]
[May 26, 2024]
Dr. Michael Y. Huang, Research Assistant, conducted an experiment using a combination of palladium and platinum electrodes. The heavy water electrolyte contained a small percentage of potassium carbonate to stabilize the reaction. Electrolysis was performed at 80 mA, with temperatures controlled between 28°C and 32°C. Measurements of voltage, current, and temperature were taken every 10 minutes. This experiment showed significant heat generation after 8 hours, with energy output exceeding input by 15%. The combination of materials proved to be effective in enhancing the cold fusion reaction.
Further analysis included monitoring gamma radiation emissions using a scintillation counter. The data revealed low levels of gamma radiation, which could be indicative of nuclear reactions occurring within the electrolytic cell. Additionally, the electrolyte solution was analyzed for transmutation products, with trace amounts of tritium and helium-3 detected, suggesting that both fusion and secondary nuclear reactions might be taking place. The palladium-platinum combination demonstrated a unique synergy, potentially opening new avenues for optimizing cold fusion systems.
[Experiment 5]
[May 27, 2024]
Dr. Laura T. Evans, Senior Chemist, explored the use of titanium electrodes in heavy water. The electrolysis process was carried out at 60 mA, with temperatures kept at a constant 30°C. Voltage and temperature readings were taken every 20 minutes. Despite a rigorous setup, the experiment yielded no significant heat excess, suggesting that titanium might not be suitable for cold fusion reactions in this context. Further studies would be needed to confirm these findings.
Additional tests included analyzing the electrolyte solution for any isotopic changes using mass spectrometry. The results indicated no significant changes in the isotopic composition, further confirming the lack of fusion activity. The titanium electrodes were examined using X-ray diffraction (XRD) to detect any structural changes, but no significant alterations were observed. These findings suggested that titanium, under the given conditions, does not facilitate cold fusion reactions effectively.
[Experiment 6]
[May 28, 2024]
In this experiment, Dr. Ahmed H. Rashid, Lead Engineer, used a hybrid electrolyte containing heavy water and a small concentration of sodium hydroxide. Palladium electrodes were again employed, with the electrolysis current set at 90 mA. The temperature was maintained at 33°C, with readings taken every 10 minutes. Over 10 hours, there was a steady increase in heat production, reaching an energy output surpassing input by 20%. The sodium hydroxide appeared to facilitate better deuterium absorption, enhancing the cold fusion process.
Neutron detection and gamma radiation monitoring were employed to further understand the reaction dynamics. The neutron counts were significantly higher than the baseline, indicating increased fusion activity. Additionally, gamma radiation levels were monitored and found to be consistent with the fusion events. The electrolyte solution was analyzed post-experiment, revealing increased helium-4 content, further supporting the occurrence of fusion reactions. The palladium electrodes exhibited significant deuterium uptake, confirmed by post-experiment isotopic analysis.
[Experiment 7]
[May 29, 2024]
Dr. Sofia B. Garcia, Junior Scientist, tested the effects of varying electrolysis currents using palladium electrodes in heavy water. The experiment began at 50 mA, incrementally increasing to 150 mA over 8 hours. Temperatures were maintained at 27°C, with constant monitoring. The results showed that higher currents significantly increased heat output, indicating a threshold current for optimal cold fusion reactions. The data suggested that a current between 100 mA and 120 mA might be ideal for future experiments.
Detailed calorimetric measurements confirmed the correlation between current and heat output, with the highest efficiency observed at 120 mA. Gas analysis indicated increased deuterium gas evolution at higher currents, suggesting enhanced fusion rates. Neutron and gamma radiation monitoring supported these findings, showing increased emissions consistent with nuclear reactions. The palladium electrodes, post-experiment, demonstrated increased lattice deformations, indicative of high deuterium loading.
[Experiment 8]
[May 30, 2024]
Dr. Jason B. White, Senior Engineer, conducted an experiment using a palladium-copper alloy for the electrodes. Heavy water with a lithium additive served as the electrolyte. Electrolysis was performed at 70 mA, with temperatures kept at 29°C. Voltage and temperature readings were taken every 15 minutes. The alloy electrodes showed enhanced heat generation, with energy output exceeding input by 18% after 9 hours. This experiment highlighted the potential benefits of using alloy materials in cold fusion processes.
The alloy composition was analyzed pre- and post-experiment to understand the material changes. SEM and energy-dispersive X-ray spectroscopy (EDS) revealed significant deuterium uptake and structural changes within the alloy matrix. Additionally, isotopic analysis of the electrolyte indicated increased helium-4 concentration, supporting the occurrence of fusion. The enhanced performance of the palladium-copper alloy suggested that material composition plays a crucial role in optimizing cold fusion reactions.
[Experiment 9]
[May 31, 2024]
Dr. Anya T. Singh, Research Fellow, explored the use of a magnetic field in conjunction with palladium electrodes. The heavy water electrolyte contained a small percentage of lithium chloride. Electrolysis was performed at 85 mA, with temperatures controlled between 30°C and 35°C. Voltage and temperature readings were taken every 10 minutes. The introduction of a magnetic field significantly increased heat output, with energy surpassing input by 22% after 7 hours. This indicated that magnetic fields might enhance the cold fusion reaction.
Further investigations included monitoring the magnetic field strength and its effect on deuterium absorption. The data suggested that the magnetic field facilitated better alignment of deuterium nuclei, enhancing fusion probabilities. Neutron and gamma radiation monitoring showed increased emissions consistent with enhanced fusion activity. The electrolyte analysis post-experiment revealed increased helium-4 and tritium content, further supporting the role of the magnetic field in enhancing cold fusion.
[Experiment 10]
[June 1, 2024]
Dr. Robert Q. Kim, Senior Physicist, tested the effects of varying temperature conditions using palladium electrodes in heavy water. The electrolysis current was set at 100 mA, with temperatures gradually increased from 25°C to 45°C over 10 hours. Voltage and temperature readings were taken every 20 minutes. The results showed a positive correlation between temperature and heat output, with maximum energy generation occurring at 40°C. This experiment underscored the importance of temperature control in optimizing cold fusion reactions.
Detailed calorimetric analysis confirmed the temperature-dependence of the fusion reaction, with a significant increase in heat output at higher temperatures. Gas analysis revealed increased deuterium gas evolution and helium-4 production, consistent with enhanced fusion rates. Neutron and gamma radiation monitoring supported these findings, showing increased emissions at higher temperatures. The palladium electrodes, post-experiment, exhibited significant structural changes, indicative of high deuterium loading and fusion activity.
[Experiment 11]
[June 2, 2024]
Dr. Maria J. Lopez, Junior Researcher, conducted an experiment using a combination of palladium and gold electrodes. Heavy water with a potassium carbonate additive served as the electrolyte. Electrolysis was performed at 95 mA, with temperatures maintained at 32°C. Voltage and temperature readings were taken every 10 minutes. The experiment showed a significant increase in heat output, with energy exceeding input by 19% after 8 hours. The combination of palladium and gold proved to be effective in enhancing cold fusion reactions.
Additional tests included monitoring the electrodes' structural integrity using SEM and XRD. The analysis revealed significant deuterium absorption and lattice changes within the palladium-gold electrodes. Isotopic analysis of the electrolyte showed increased helium-4 and tritium content, supporting the occurrence of fusion reactions. Neutron and gamma radiation monitoring confirmed increased emissions, consistent with nuclear activity. The palladium-gold combination demonstrated a unique synergy, enhancing the cold fusion process.
[Experiment 12]
[June 3, 2024]
Dr. Benjamin K. Park, Research Assistant, explored the use of high-pressure conditions with palladium electrodes in heavy water. Electrolysis was performed at 75 mA, with the pressure increased to 3 atmospheres. Temperatures were maintained at 28°C, with readings taken every 15 minutes. The experiment yielded a noticeable increase in heat output, with energy surpassing input by 21% after 6 hours. The high-pressure conditions appeared to enhance deuterium absorption, improving the cold fusion reaction.
Further analysis included monitoring the gas output under high pressure, revealing increased deuterium and helium-4 levels. Neutron and gamma radiation emissions were also monitored, showing significant increases consistent with enhanced fusion activity. Post-experiment analysis of the palladium electrodes using XRD and SEM revealed substantial lattice deformations, indicative of high deuterium loading and fusion activity. The high-pressure conditions proved to be a crucial factor in optimizing cold fusion reactions.
[Experiment 13]
[June 4, 2024]
Dr. Clara R. Martinez, Senior Scientist, tested the effects of varying electrolyte compositions using palladium electrodes. The experiment involved heavy water with different concentrations of lithium chloride. Electrolysis was performed at 80 mA, with temperatures kept at 30°C. Voltage and temperature readings were taken every 10 minutes. The results showed that a 0.1 M concentration of lithium chloride produced the highest heat output, with energy exceeding input by 17% after 9 hours. This experiment highlighted the importance of electrolyte composition in cold fusion processes.
Detailed analysis included monitoring the deuterium absorption rates and gas evolution. The optimal lithium chloride concentration facilitated better deuterium uptake, confirmed by increased helium-4 production in the electrolyte. Neutron and gamma radiation monitoring showed increased emissions at the optimal concentration, consistent with enhanced fusion activity. Post-experiment analysis of the palladium electrodes revealed significant structural changes, indicative of high deuterium loading and effective fusion reactions.
[Experiment 14]
[June 5, 2024]
Dr. John O. Lewis, Lead Engineer, conducted an experiment using a combination of palladium and titanium electrodes. Heavy water with a sodium hydroxide additive served as the electrolyte. Electrolysis was performed at 85 mA, with temperatures controlled between 29°C and 34°C. Voltage and temperature readings were taken every 10 minutes. The experiment showed significant heat generation, with energy output surpassing input by 20% after 8 hours. The combination of materials and electrolyte composition proved effective in enhancing cold fusion reactions.
Additional tests included monitoring the structural changes in the electrodes using SEM and XRD. The analysis revealed substantial deuterium absorption and lattice changes within the palladium-titanium electrodes. Isotopic analysis of the electrolyte showed increased helium-4 and tritium content, supporting the occurrence of fusion reactions. Neutron and gamma radiation monitoring confirmed increased emissions, consistent with nuclear activity. The palladium-titanium combination demonstrated a unique synergy, enhancing the cold fusion process.
[Experiment 15]
[June 6, 2024]
Dr. Lisa M. Cheng, Junior Scientist, explored the use of alternating current (AC) electrolysis with palladium electrodes in heavy water. The AC current was set at a frequency of 50 Hz and an amplitude of 90 mA. Temperatures were maintained at 27°C, with readings taken every 10 minutes. The experiment yielded significant heat output, with energy exceeding input by 18% after 7 hours. The use of AC current appeared to enhance deuterium absorption and improve cold fusion reactions.
Detailed analysis included monitoring the frequency and amplitude effects on fusion rates. The optimal AC parameters facilitated better deuterium alignment, confirmed by increased helium-4 production in the electrolyte. Neutron and gamma radiation monitoring showed increased emissions at the optimal AC settings, consistent with enhanced fusion activity. Post-experiment analysis of the palladium electrodes revealed significant structural changes, indicative of high deuterium loading and effective fusion reactions.
[Experiment 16]
[June 7, 2024]
Dr. David R, Morgan, Senior Chemist, tested the effects of varying pressure conditions using palladium electrodes in heavy water. Electrolysis was performed at 70 mA, with the pressure gradually increased to 4 atmospheres. Temperatures were maintained at 30°C, with readings taken every 15 minutes. The experiment showed a noticeable increase in heat output, with energy surpassing input by 22% after 6 hours. The high-pressure conditions appeared to enhance deuterium absorption, improving the cold fusion reaction.
Further analysis included monitoring the gas output under high pressure, revealing increased deuterium and helium-4 levels. Neutron and gamma radiation emissions were also monitored, showing significant increases consistent with enhanced fusion activity. Post-experiment analysis of the palladium electrodes using XRD and SEM revealed substantial lattice deformations, indicative of high deuterium loading and fusion activity. The high-pressure conditions proved to be a crucial factor in optimizing cold fusion reactions.
[Experiment 17]
[June 8, 4]
Dr. Olivia P. Rodriguez, Research Fellow, conducted an experiment using palladium electrodes and a magnetic field in heavy water. The electrolyte contained a small percentage of lithium carbonate. Electrolysis was performed at 90 mA, with temperatures controlled between 31°C and 36°C. Voltage and temperature readings were taken every 10 minutes. The experiment yielded significant heat generation, with energy output surpassing input by 23% after 7 hours. The combination of materials, magnetic field, and electrolyte composition proved effective in enhancing cold fusion reactions.
Detailed investigations included monitoring the magnetic field strength and its effect on deuterium absorption. The data suggested that the magnetic field facilitated better alignment of deuterium nuclei, enhancing fusion probabilities. Neutron and gamma radiation monitoring showed increased emissions consistent with enhanced fusion activity. The electrolyte analysis post-experiment revealed increased helium-4 and tritium content, further supporting the role of the magnetic field in enhancing cold fusion.
[Experiment 18]
[June 9, 2024]
Dr. Kevin B. Wright, Senior Physicist, tested the effects of varying electrolysis currents using palladium electrodes in heavy water. The experiment began at 60 mA, incrementally increasing to 160 mA over 10 hours. Temperatures were maintained at 28°C, with constant monitoring. The results showed that higher currents significantly increased heat output, indicating a threshold current for optimal cold fusion reactions. The data suggested that a current between 110 mA and 130 mA might be ideal for future experiments.
Detailed calorimetric measurements confirmed the correlation between current and heat output, with the highest efficiency observed at 120 mA. Gas analysis indicated increased deuterium gas evolution at higher currents, suggesting enhanced fusion rates. Neutron and gamma radiation monitoring supported these findings, showing increased emissions consistent with nuclear reactions. The palladium electrodes, post-experiment, demonstrated increased lattice deformations, indicative of high deuterium loading.
[Experiment 19]
[June 10, 2024]
Dr. Natasha R. Ivanov, Junior Researcher, explored the use of a hybrid electrolyte containing heavy water and a small concentration of potassium hydroxide. Palladium electrodes were again employed, with the electrolysis current set at 95 mA. The temperature was maintained at 32°C, with readings taken every 10 minutes. Over 10 hours, there was a steady increase in heat production, reaching an energy output surpassing input by 20%. The potassium hydroxide appeared to facilitate better deuterium absorption, enhancing the cold fusion process.
Neutron detection and gamma radiation monitoring were employed to further understand the reaction dynamics. The neutron counts were significantly higher than the baseline, indicating increased fusion activity. Additionally, gamma radiation levels were monitored and found to be consistent with the fusion events. The electrolyte solution was analyzed post-experiment, revealing increased helium-4 content, further supporting the occurrence of fusion reactions. The palladium electrodes exhibited significant deuterium uptake, confirmed by post-experiment isotopic analysis.
[Experiment 20]
[June 11, 2024]
Dr. Hiroshi N. Tanaka, Chief Scientist, conducted an experiment using a combination of palladium and silver electrodes. Heavy water with a lithium chloride additive served as the electrolyte. Electrolysis was performed at 100 mA, with temperatures maintained at 30°C. Voltage and temperature readings were taken every 10 minutes. The experiment showed significant heat generation, with energy output surpassing input by 25% after 8 hours. The combination of palladium and silver, along with the lithium chloride electrolyte, proved to be highly effective in enhancing cold fusion reactions.
Additional investigations included monitoring the electrodes' structural integrity using SEM and XRD. The analysis revealed substantial deuterium absorption and lattice changes within the palladium-silver electrodes. Isotopic analysis of the electrolyte showed increased helium-4 and tritium content, supporting the occurrence of fusion reactions. Neutron and gamma radiation monitoring confirmed increased emissions, consistent with nuclear activity. The palladium-silver combination demonstrated a unique synergy, enhancing the cold fusion process significantly.
"""

def parse_experiments():
    # Split the text using the experiment identifier as a delimiter
    experiments = get_coldf_experiment_text().split('[Experiment')
    
    # Remove empty strings and reformat each experiment
    experiments = ['[Experiment' + exp.strip() for exp in experiments if exp.strip()]
    
    return experiments

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import numpy as np
import chromadb


def get_context_encoder_tokenizer():
    """ Load the DPR context encoder and tokenizer. """
    context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
    return context_encoder, context_tokenizer

def init_chroma_db(store_name:str="documents"):
    """ Initialize ChromaDB client. """
    chroma_client = chromadb.PersistentClient(path="./cromadb")
    vector_store = chroma_client.get_or_create_collection(store_name)
    return chroma_client, vector_store

def chunk_embed_text(text, chunk_size, overlap_size, context_encoder, context_tokenizer):
    """Chunk the text into overlapping segments and generate embeddings for the text using a transformer model.."""

    chunks = []
    embeddings = []
    ids = []
    ct = 0
    for chunk in text:
        chunks.append(chunk)
        inputs = context_tokenizer(chunk, return_tensors='pt')
        embedding = context_encoder(**inputs).pooler_output.detach().numpy()[0].tolist()
        embeddings.append(embedding)

        print(chunk, embedding, ct)
        
        ids.append(f"id_{str(ct)}")
        ct += 1


    # chunks = []
    # embeddings = []
    # ids = []
    # start = 0
    # ct = 0
    # while start < len(text):
    #     end = start + chunk_size
    #     chunk = text[start:end]
    #     chunks.append(chunk)
    #     start += chunk_size - overlap_size

    #     inputs = context_tokenizer(chunk, return_tensors='pt')
    #     embedding = context_encoder(**inputs).pooler_output.detach().numpy()[0].tolist()
    #     embeddings.append(embedding)

    #     print(chunk, embedding, ct)
        
    #     ids.append(f"id_{str(ct)}")
    #     ct += 1

    # print(f"------------====>{embeddings}")
    return chunks, embeddings, ids


def preprocess_text_to_chroma(text, vector_store, chunk_size, overlap_size, context_encoder, context_tokenizer): 
    """Process text and store chunks in ChromaDB."""
    
    chunks, embeddings, ids = chunk_embed_text(text, 
                                               chunk_size, 
                                               overlap_size, 
                                               context_encoder=context_encoder, 
                                               context_tokenizer=context_tokenizer)
    vector_store.add(documents=chunks, embeddings=embeddings, ids=ids)
    




In [ ]:
# Configuration
chunk_size = 500  # Define your chunk size, each experiment has more or less 
overlap_size = 50  # Define your overlap size

# Example text corpus
text = parse_experiments()

context_encoder, context_tokenizer = get_context_encoder_tokenizer()
chroma_client, vector_store = init_chroma_db("documents")
preprocess_text_to_chroma(text, 
                          vector_store, 
                          chunk_size, 
                          overlap_size, 
                          context_tokenizer=context_tokenizer, 
                          context_encoder=context_encoder)

In [5]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import numpy as np
import chromadb


def init_chroma_db(store_name:str="documents"):
    """ Initialize ChromaDB client. """
    chroma_client = chromadb.PersistentClient(path="./cromadb")
    vector_store = chroma_client.get_or_create_collection(store_name)
    return chroma_client, vector_store

def chunk_embed_text(text):
    """Generate embeddings for the text using a transformer model.."""

    chunks = []
    ids = []
    ct = 0
    for chunk in text:
        chunks.append(chunk)
        ids.append(f"id_{str(ct)}")
        ct += 1

    return chunks, ids


def preprocess_text_to_chroma(text, vector_store): 
    """Process text and store chunks in ChromaDB."""
    
    chunks, ids = chunk_embed_text(text)
    vector_store.add(documents=chunks, ids=ids)
    
# Example text corpus
text = parse_experiments()

chroma_client, vector_store = init_chroma_db("documents")
preprocess_text_to_chroma(text, 
                          vector_store)


Add of existing embedding ID: id_0
Add of existing embedding ID: id_1
Add of existing embedding ID: id_2
Add of existing embedding ID: id_3
Add of existing embedding ID: id_4
Add of existing embedding ID: id_5
Add of existing embedding ID: id_6
Add of existing embedding ID: id_7
Add of existing embedding ID: id_8
Add of existing embedding ID: id_9
Add of existing embedding ID: id_10
Add of existing embedding ID: id_11
Add of existing embedding ID: id_12
Add of existing embedding ID: id_13
Add of existing embedding ID: id_14
Add of existing embedding ID: id_15
Add of existing embedding ID: id_16
Add of existing embedding ID: id_17
Add of existing embedding ID: id_18
Add of existing embedding ID: id_19
Insert of existing embedding ID: id_0
Insert of existing embedding ID: id_1
Insert of existing embedding ID: id_2
Insert of existing embedding ID: id_3
Insert of existing embedding ID: id_4
Insert of existing embedding ID: id_5
Insert of existing embedding ID: id_6
Insert of existing embed

{'ids': [['id_0', 'id_8', 'id_3', 'id_16', 'id_19', 'id_18', 'id_10', 'id_5', 'id_11', 'id_15']], 'distances': [[0.8525894004258584, 0.9353905196358957, 0.9482485195863827, 0.9590379392588978, 0.9841376880261145, 1.0198892389075616, 1.0525335040210608, 1.0530792117099146, 1.054134360354219, 1.0597632357084095]], 'metadatas': [[None, None, None, None, None, None, None, None, None, None]], 'embeddings': None, 'documents': [['[Experiment1]\n[May 23, 2024]\nThe first experiment focused on using palladium electrodes submerged in heavy water (deuterium oxide, D2O). Dr. Emily Jensen, Senior Physicist, led this trial. The procedure involved electrolysis at a constant current of 50 mA, aiming to induce cold fusion within the palladium lattice. Throughout the 12-hour process, temperatures were carefully monitored, maintaining a steady 25°C. Voltage readings were recorded every hour to observe any anomalies indicating fusion events. The experiment yielded promising preliminary results with minor 

In [24]:
question = "What were the key findings in the last successful cold fusion experiment?"
# question = "What is the color of palladium?"
results = vector_store.query(query_texts=question, n_results=5)

documents = "\n".join(results['documents'][0])

prompt = f"""DOCUMENT:
{documents}

QUESTION:
{question}

INSTRUCTIONS:
Answer the users QUESTION using the DOCUMENT text above.
Keep your answer ground in the facts of the DOCUMENT.
If the DOCUMENT doesn’t contain the facts to answer the QUESTION return 'NONE'"""

In [25]:
from ollama import Client
host = ""
model = "llama3"

system_message = {"role": "system", "content": prompt}
messages = [system_message]

response = Client(host=host).chat(model=model, messages=messages, options= {"seed": 42, "top_p": 0.9, "temperature": 0 })

In [26]:
response

{'model': 'llama3',
 'created_at': '2024-06-13T16:04:55.040275131Z',
 'message': {'role': 'assistant',
  'content': 'The key finding in the last successful cold fusion experiment (Experiment20) was that the combination of palladium and silver electrodes, with a lithium chloride electrolyte, proved to be highly effective in enhancing cold fusion reactions. The experiment showed significant heat generation, with energy output surpassing input by 25% after 8 hours.'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 20054003340,
 'load_duration': 1495696,
 'prompt_eval_count': 1125,
 'prompt_eval_duration': 7309602000,
 'eval_count': 66,
 'eval_duration': 12585410000}

In [ ]:
# Configuration
chunk_size = 500  # Define your chunk size
overlap_size = 50  # Define your overlap size

# Example text corpus
text = get_coldf_experiment_text()
# text = """
# CHAPTER I
# A SHIFTING REEF
# The year 1866 was signalised by a remarkable incident, a mysterious and puzzling phenomenon, which doubtless no one has yet forgotten. Not to mention rumours which agitated the maritime population and excited the public mind, even in the interior of continents, seafaring men were particularly excited. Merchants, common sailors, captains of vessels, skippers, both of Europe and America, naval officers of all countries, and the Governments of several states on the two continents, were deeply interested in the matter.

# For some time past, vessels had been met by “an enormous thing,” a long object, spindle-shaped, occasionally phosphorescent, and infinitely larger and more rapid in its movements than a whale.

# The facts relating to this apparition (entered in various log-books) agreed in most respects as to the shape of the object or creature in question, the untiring rapidity of its movements, its surprising power of locomotion, and the peculiar life with which it seemed endowed. If it was a cetacean, it surpassed in size all those hitherto classified in science. Taking into consideration the mean of observations made at divers times,—rejecting the timid estimate of those who assigned to this object a length of two hundred feet, equally with the exaggerated opinions which set it down as a mile in width and three in length,—we might fairly conclude that this mysterious being surpassed greatly all dimensions admitted by the ichthyologists of the day, if it existed at all. And that it did exist was an undeniable fact; and, with that tendency which disposes the human mind in favour of the marvellous, we can understand the excitement produced in the entire world by this supernatural apparition. As to classing it in the list of fables, the idea was out of the question.

# On the 20th of July, 1866, the steamer Governor Higginson, of the Calcutta and Burnach Steam Navigation Company, had met this moving mass five miles off the east coast of Australia. Captain Baker thought at first that he was in the presence of an unknown sandbank; he even prepared to determine its exact position, when two columns of water, projected by the inexplicable object, shot with a hissing noise a hundred and fifty feet up into the air. Now, unless the sandbank had been submitted to the intermittent eruption of a geyser, the Governor Higginson had to do neither more nor less than with an aquatic mammal, unknown till then, which threw up from its blow-holes columns of water mixed with air and vapour.

# Similar facts were observed on the 23rd of July in the same year, in the Pacific Ocean, by the Columbus, of the West India and Pacific Steam Navigation Company. But this extraordinary cetaceous creature could transport itself from one place to another with surprising velocity; as, in an interval of three days, the Governor Higginson and the Columbus had observed it at two different points of the chart, separated by a distance of more than seven hundred nautical leagues.

# Fifteen days later, two thousand miles farther off, the Helvetia, of the Compagnie-Nationale, and the Shannon, of the Royal Mail Steamship Company, sailing to windward in that portion of the Atlantic lying between the United States and Europe, respectively signalled the monster to each other in 42° 15′ N. lat. and 60° 35′ W. long. In these simultaneous observations they thought themselves justified in estimating the minimum length of the mammal at more than three hundred and fifty feet, as the Shannon and Helvetia were of smaller dimensions than it, though they measured three hundred feet over all.

# Now the largest whales, those which frequent those parts of the sea round the Aleutian, Kulammak, and Umgullich islands, have never exceeded the length of sixty yards, if they attain that.

# These reports arriving one after the other, with fresh observations made on board the transatlantic ship Pereire, a collision which occurred between the Etna of the Inman line and the monster, a procès verbal directed by the officers of the French frigate Normandie, a very accurate survey made by the staff of Commodore Fitz-James on board the Lord Clyde, greatly influenced public opinion. Light-thinking people jested upon the phenomenon, but grave practical countries, such as England, America, and Germany, treated the matter more seriously.

# In every place of great resort the monster was the fashion. They sang of it in the cafés, ridiculed it in the papers, and represented it on the stage. All kinds of stories were circulated regarding it. There appeared in the papers caricatures of every gigantic and imaginary creature, from the white whale, the terrible “Moby Dick” of hyperborean regions, to the immense kraken whose tentacles could entangle a ship of five hundred tons, and hurry it into the abyss of the ocean. The legends of ancient times were even resuscitated, and the opinions of Aristotle and Pliny revived, who admitted the existence of these monsters, as well as the Norwegian tales of Bishop Pontoppidan, the accounts of Paul Heggede, and, last of all, the reports of Mr. Harrington (whose good faith no one could suspect), who affirmed that, being on board the Castillan, in 1857, he had seen this enormous serpent, which had never until that time frequented any other seas but those of the ancient “Constitutionnel.”

# Then burst forth the interminable controversy between the credulous and the incredulous in the societies of savants and the scientific journals. “The question of the monster” inflamed all minds. Editors of scientific journals, quarrelling with believers in the supernatural, spilled seas of ink during this memorable campaign, some even drawing blood; for, from the sea-serpent they came to direct personalities.

# For six months war was waged with various fortune in the leading articles of the Geographical Institution of Brazil, the Royal Academy of Science of Berlin, the British Association, the Smithsonian Institution of Washington, in the discussions of the “Indian Archipelago,” of the Cosmos of the Abbé Moigno, in the Mittheilungen of Petermann, in the scientific chronicles of the great journals of France and other countries. The cheaper journals replied keenly and with inexhaustible zest. These satirical writers parodied a remark of Linnæus, quoted by the adversaries of the monster, maintaining “that nature did not make fools,” and adjured their contemporaries not to give the lie to nature, by admitting the existence of krakens, sea-serpents, “Moby Dicks,” and other lucubrations of delirious sailors. At length an article in a well-known satirical journal by a favourite contributor, the chief of the staff, settled the monster, like Hippolytus, giving it the death-blow amidst an universal burst of laughter. Wit had conquered science.

# During the first months of the year 1867 the question seemed buried, never to revive, when new facts were brought before the public. It was then no longer a scientific problem to be solved, but a real danger seriously to be avoided. The question took quite another shape. The monster became a small island, a rock, a reef, but a reef of indefinite and shifting proportions.

# On the 5th of March, 1867, the Moravian, of the Montreal Ocean Company, finding herself during the night in 27° 30′ lat. and 72° 15′ long., struck on her starboard quarter a rock, marked in no chart for that part of the sea. Under the combined efforts of the wind and its four hundred horse-power, it was going at the rate of thirteen knots. Had it not been for the superior strength of the hull of the Moravian, she would have been broken by the shock and gone down with the 237 passengers she was bringing home from Canada.

# The accident happened about five o’clock in the morning, as the day was breaking. The officers of the quarter-deck hurried to the after-part of the vessel. They examined the sea with the most scrupulous attention. They saw nothing but a strong eddy about three cables’ length distant, as if the surface had been violently agitated. The bearings of the place were taken exactly, and the Moravian continued its route without apparent damage. Had it struck on a submerged rock, or on an enormous wreck? they could not tell; but on examination of the ship’s bottom when undergoing repairs, it was found that part of her keel was broken.

# This fact, so grave in itself, might perhaps have been forgotten like many others if, three weeks after, it had not been re-enacted under similar circumstances. But, thanks to the nationality of the victim of the shock, thanks to the reputation of the company to which the vessel belonged, the circumstance became extensively circulated.

# The 13th of April, 1867, the sea being beautiful, the breeze favourable, the Scotia, of the Cunard Company’s line, found herself in 15° 12′ long. and 45° 37′ lat. She was going at the speed of thirteen knots and a half.

# At seventeen minutes past four in the afternoon, whilst the passengers were assembled at lunch in the great saloon, a slight shock was felt on the hull of the Scotia, on her quarter, a little aft of the port-paddle.

# The Scotia had not struck, but she had been struck, and seemingly by something rather sharp and penetrating than blunt. The shock had been so slight that no one had been alarmed, had it not been for the shouts of the carpenter’s watch, who rushed on to the bridge, exclaiming, “We are sinking! we are sinking!” At first the passengers were much frightened, but Captain Anderson hastened to reassure them. The danger could not be imminent. The Scotia, divided into seven compartments by strong partitions, could brave with impunity any leak. Captain Anderson went down immediately into the hold. He found that the sea was pouring into the fifth compartment; and the rapidity of the influx proved that the force of the water was considerable. Fortunately this compartment did not hold the boilers, or the fires would have been immediately extinguished. Captain Anderson ordered the engines to be stopped at once, and one of the men went down to ascertain the extent of the injury. Some minutes afterwards they discovered the existence of a large hole, of two yards in diameter, in the ship’s bottom. Such a leak could not be stopped; and the Scotia, her paddles half submerged, was obliged to continue her course. She was then three hundred miles from Cape Clear, and after three days’ delay, which caused great uneasiness in Liverpool, she entered the basin of the company.

# The engineers visited the Scotia, which was put in dry dock. They could scarcely believe it possible; at two yards and a half below water-mark was a regular rent, in the form of an isosceles triangle. The broken place in the iron plates was so perfectly defined that it could not have been more neatly done by a punch. It was clear, then, that the instrument producing the perforation was not of a common stamp; and after having been driven with prodigious strength, and piercing an iron plate 1-3/8 inches thick, had withdrawn itself by a retrograde motion truly inexplicable.

# Such was the last fact, which resulted in exciting once more the torrent of public opinion. From this moment all unlucky casualties which could not be otherwise accounted for were put down to the monster. Upon this imaginary creature rested the responsibility of all these shipwrecks, which unfortunately were considerable; for of three thousand ships whose loss was annually recorded at Lloyd’s, the number of sailing and steam ships supposed to be totally lost, from the absence of all news, amounted to not less than two hundred!

# Now, it was the “monster” who, justly or unjustly, was accused of their disappearance, and, thanks to it, communication between the different continents became more and more dangerous. The public demanded peremptorily that the seas should at any price be relieved from this formidable cetacean. 
# """


context_encoder, context_tokenizer = get_context_encoder_tokenizer()
chroma_client, vector_store = init_chroma_db("documents")
preprocess_text_to_chroma(text, vector_store, chunk_size, overlap_size, context_tokenizer=context_tokenizer, context_encoder=context_encoder)



In [ ]:
query_text = "Who is Alexandre Dumas?"
# _, query_embeddings, _   = chunk_embed_text(query_text, chunk_size, overlap_size,context_encoder, context_tokenizer)

# results = vector_store.query(query_embeddings, n_results=3)
results = vector_store.query(query_texts=query_text, n_results=3)

print(results)

In [ ]:
# CODE 1


from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

text = "apple"

context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

inputs = context_tokenizer(text, return_tensors='pt')
embeddings = context_encoder(**inputs).pooler_output.detach().numpy()[0].tolist()
print(f"Embeddings:{embeddings}")
print(f"Lenght embeddings:{len(embeddings)}")


In [ ]:
inputs

In [ ]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
input_ids = tokenizer(text, return_tensors="pt")["input_ids"]
embeddings = model(input_ids).pooler_output

print(embeddings.shape)